<a href="https://colab.research.google.com/github/cmlins/recuperacaoDeInformacao/blob/master/wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests as rq
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Funções auxiliares

In [0]:
 # Create DataFrame

def create_df(info_book_list):
  cols = ['Título', 'Autor', 'Nº Páginas', 'Editora', 'Idioma', 'Acabamento', 'ISBN']

  book_pd = pd.DataFrame([info_book_list],\
                        columns=cols)

  return (book_pd)

Medir:

– Total	de	extrações possíves:	N	
– Total	de	pares	extraídos pelo sistema:	E	
– Total	de	pares	extraídos corretamente:	C	

Computar:

– Recall	=	C/N	
– Precision	=	C/E	
– F-Measure=	2*(Recall * Precision)/(Recall + Precision)

In [0]:
# Print Recall, Precision and F-Measure

def analises(url_list, df):

  N = len(url_list)

  E = df.shape[0]

  # create a column with the sum of cells with data in each row
  df['count'] = (df != "").sum(1)

  # create a list with the values from column 'count'
  c_list = []
  for index, row in df.iterrows(): 
    c_list.append(row['count'])

  # create a list with unique values  
  c_list_unique = np.unique(c_list)

  # create a list of tuples 
  c_list_count = []
  for value in c_list_unique:
    c_list_count.append((value, c_list.count(value)))



  for tupla in c_list_count:
    for i in range(len(tupla)):
      if tupla[0] == 7:
        c = tupla[1]
      else:
        c = 0

  C = c

  recall = C / N

  precision = C / E

  f_mult = (recall * precision)

  f_sum = sum([recall, precision])

  try:
    f_div = f_mult / f_sum
  except ZeroDivisionError:
    f_div = 0

  f_measure = round(2 * f_div, 3)

  print('Recall: {} \nPrecision: {} \nF_Measure = {} \nC = {} \nN = {} \nE = {}'.format(recall, precision, f_measure, C, N, E))

## Sites Livros

1.   https://www.amazon.com.br/
2.   https://www.saraiva.com.br/
3.   https://www.travessa.com.br/
4.   https://www.traca.com.br/
5.   https://www.ciadoslivros.com.br/
6.   https://www.extra.com.br/livros/
7.   https://www.livrariacultura.com.br
8.   https://www.disal.com.br/
9.   https://www.americanas.com.br/
10.  https://www.submarino.com.br/


## URLS

In [0]:
amazon_urls = [ r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/1808.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Box - As Melhores Aventuras Do Sítio Do Picapau Amarelo Monteiro Lobato Amazon.com.br.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Cotoco. O Diário De Um Garoto De 13 Anos - Livros na Amazon Brasil- 9788598078854.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Doctor Who, Neil Gaiman e vários - Livros na Amazon.com.br.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Freud (1900) A interpretação dos sonhos Obras completas volume 4 - 9788535932218 - Livros na Amazon Brasil.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Good Omens Belas Maldições - Livros na Amazon Brasil- 9788528624021.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Os Mitos de Cthulhu - Volume Único (Exclusivo Amazon) Livros.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Redes de Computadores e a Internet Uma Abordagem Top-Down - 9788581436777 - Livros na Amazon Brasil.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Amazon/Positivas/Sobotta - Atlas de Anatomia Humana - 3 Volumes - 9788527732376 - Livros na Amazon Brasil.html']

In [0]:
saraiva_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Alice No País Das Maravilhas (Classic Edition) - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Steve Jobs - A Biografia - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Vade Mecum Compacto - Brochura - 21a Ed. 2019 - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/It - A Coisa - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Como Fazer Amigos e Influenciar Pessoas - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/O Guia do Mochileiro Das Galáxias - Série o Mochileiro Das Galáxias - Vol. 1 - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/O Livrinho do Bebê - Animais, Palavras, Contrários, Cores e Formas - Saraiva.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Saraiva/Positivas/Star Wars - O Arquivo Rebelde - Saraiva.html']

In [0]:
travessa_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/A COR DA MAGIA - Terry Pratchett - Livro.html',                
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/CYNDI MINHA HISTORIA - Cyndi Lauper Jancee Dunn - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/D. PEDRO II A HISTORIA NAO CONTADA - Paulo Rezzutti - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/JAPONES PARA LEIGOS - Eriko Sato - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/LONELY PLANET ARGENTINA (INCLUI URUGUAI) - Globo Livros - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/MADELINE FINN E OS CÃES DO ABRIGO - Lisa Papp - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/O PODER DA AUTORRESPONSABILIDADE A FERRAMENTA COMPROVADA QUE GERA ALTA PERFORMANCE E RESULTADOS EM POUCO TEMPO - Paulo Vieira - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/CRONICAS PARA LER EM QUALQUER LUGAR - Vários (ver informações no detalhe) - Livro.html',                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/SO PARA UM ALIMENTAÇAO SAUDAVEL PARA QUEM MORA SOZINHO - Rita Lobo - Livro.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Travessa/Positivas/VIDAS SECAS - 140aED.(2019) - Graciliano Ramos - Livro.html']

In [0]:
traca_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/A Casa Dos Budas Ditosos - João Ubaldo Ribeiro - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Cavaleiros Do Zodíaco Vol 1 - Masami Kurumada - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Cem Anos De Solidão - Gabriel Garcia Marquez - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Crianças Famosas  Schubert - Ann Rachlin E Susan Hellard - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Indiozinhos - Mônica Haibara - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Livro Da Esperança - Francisco Cândido Xavier - Emmanuel - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Mozart Don Giovanni - A Vida De Mozart - Lorenzo Da Ponte - Henry Stendhal - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/O Colapso Do Universo - Isaac Asimov - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Políticas Territoriais Na Amazônia - Neli Aparecida De Mello - Traça Livraria e Sebo.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Traça/Positivas/Relações Exteriores Do Brasil Contemporâneo - Danielly Silva Ramos Becard - Traça Livraria e Sebo.html']

In [0]:
cia_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Afirmar-se com Nietzsche - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Audácia Dessa Mulher, A - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Avaliação - Mito e Desafio - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Breve História do Feminismo no Contexto Euro-americano, Uma - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Corpo Fala, O - A Linguagem Silenciosa da Comunicação Não Verbal - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Cura do Ciúme, A - Aprenda a Confiar, Supere a Possessividade e Salve Seu Relacionamento - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Dom Casmurro - Coleção Grandes Mestres da Literatura Brasileira - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Em Busca do Texto Perfeito - Questões Contemporâneas de Edição, Preparação e Revisão Textual - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Manual do Arquiteto Descalço - Capa Dura - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Feminismo É Feminino?, O - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Gramática de Espanhol Para Brasileiros - Cia. dos Livros.html',
            r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Cia dos Livros/Positivas/Livro Nem Só De Ciência Se Faz A Cura - O Que Os Pacientes Me Ensinaram - Cia. dos Livros.html']

In [0]:
extra_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Aromaterapia e as Emoções - Como Usar Óleos Essenciais Para Equilibrar o Corpo e a Mente - Especialidades Médicas no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Once Upon A Time - Irmãos Grimm - Juvenil no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Os Miseráveis - Walcyr Carrasco - Juvenil no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Panelinha - Receitas Que Funcionam - Rita Lobo - Culinária no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - A Pequena Filosofia da Mafalda - Vamos à Sopa - Joaquim Salvador Lavado (Quino) - Infantis no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - A Seleção_ 35 Garotas e uma Coroa - Volume 1 - Kiera Cass - Juvenil no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Como Eu Era Antes de Você - Jojo Moyes - Romance no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Elas Podem... e Devem - o Livro que vai Mexer com Você - Flaviane Brandemberg - Sexologia no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Em Busca de Sentido - Um Psicólogo no Campo de Concentração - Psicologia no Extra.com.br.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Extra/Positivas/Livro - Eu, Fernando Pessoa - em Quadrinhos - Susana Ventura - Quadrinhos Adultos no Extra.com.br.html']

In [0]:
cultura_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - MAIS ESPERTO QUE O DIABO - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - O CAIBALION - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - O PODER DO HÁBITO - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - QUEM PENSA ENRIQUECE - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro - SOCIEDADE DO CANSAÇO - Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro GESTALT-TERAPIA - FUNDAMENTOS EPISTEMOLOGICOS | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro INTRODUÇAO A ALGEBRA LINEAR COM APLICAÇOES | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro O DIÁRIO DE ANNE FRANK | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro O MUNDO SEGUNDO FELIPE NETO | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro O PEQUENO PRINCIPE (LIVRO DE BOLSO) | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro PRÓLOGO, ATO, EPÍLOGO | Livraria Cultura.html',
                r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Livaria Cultura/Positivas/Livro QUEM PENSA ENRIQUECE | Livraria Cultura.html']

In [0]:
disal_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Ambiente E Os Processos De Maturacao, O - 9788573074567.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Autorretrato E Outras Crônicas - 9788501112330.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Aventuras Na Netoland Com Luccas Neto - 9788555461002.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Exames Laboratoriais E Diagnosticos Em Enfermagem - 9o Ed - 9788527726634.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Fisiologia Endocrina (lange) - 4o Ed - 9788580553918.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Gestao Hospitalar - Para Uma Administracao Eficaz - 9788527733298.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Gramatica Da Lingua Portuguesa Para Leigos - 9788550803395.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Introducao A Analise Matematica - 9788521201687.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Michaelis Minidicionario Alemao - Alemao-portugues - Portugues-alemao - 9788506078563.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Michaelis Minidicionario Espanhol - Espanhol-portugues _ Portugues-espanhol - 9788506078532.html',
              r'/content/drive/My Drive/9periodo/Recuperação de Informação/SITES BAIXADOS (Cris)/Disal/Positivas/Michaelis Minidicionario Frances - Frances-portugues _ Portugues-frances - 3a Ed - 9788506078549.html']

In [0]:
americanas_urls = [r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - A Cinco Passos de Você nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - A Corrente nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - As 13 Maldições - Edição Econômica nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Crônicas Saxônicas: A Guerra do Lobo nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Eu Sou Eric Zimmerman nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Herdeiro de Sevenwaters - Coleção Sevenwaters - Vol. 4 nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Historia Da Filosofia Crista nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Medicina dos Horrores nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - O Cadete e o Capitão nas Lojas Americanas.com.html',
                   r'/content/drive/My Drive/9periodo/Recuperação de Informação/Americanas/Positivas/Livro - Paixão Libertadora - Desejo Proibido - Vol. 2 nas Lojas Americanas.com.html']

In [0]:
submarino_urls = [  r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/LIvro - Luccas Neto em Os Aventureiros no Submarino.com.html',
                    r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - A Dama Mais Apaixonada no Submarino.com.html',
                    r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - O Labirinto do Fauno no Submarino.com.html',
                    r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Sandman - Edição Definitiva no Submarino.com.html',
                    r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - O Rei Leão Livro Oficial com a História do Filme no Submarino.com.html',
                    r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Akira no Submarino.com.html',
                    r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Destinos Divididos no Submarino.com.html',
                    r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Melhor que a Encomenda no Submarino.com.html',
                    r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Um Carinho na Alma + 3 Cordéis do Autor no Submarino.com.html',
                    r'/content/drive/My Drive/9periodo/Recuperação de Informação/Submarino/Positivas/Livro - Seja Foda! no Submarino.com.html']

In [0]:
urls = amazon_urls + saraiva_urls + travessa_urls + traca_urls + cia_urls + extra_urls + cultura_urls + disal_urls + americanas_urls + submarino_urls

## 1. AMAZON

In [0]:
def amazon_single(site):

  soup = bs(open(site),"html.parser")

  book = soup.find('div', {'class':'content'}).find_all('li')[:5]
  
  infos = soup.find_all('ul')
  infos = [info.text.split('\n') for info in infos]
  infos = [info for info in infos]
  infos = [info[i].split(':') for info in infos for i in range(len(info))]


  infos2 = soup.find('h1', {'id':'title'}).find_all('span')
  infos2 = [info.text.split('\n') for info in infos2]
  infos2 = [info for info in infos2]

  n_pages = book[0].contents[1].strip().split(' ')[0]
  
  publishing = book[1].contents[1].strip().split(';')[0]
  
  language = book[2].contents[1].strip()
  
  isbn = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'ISBN' or info[i] == 'ISBN 13' or info[i] == 'ISBN-13']
  
  title = infos2[0][0]

  language = infos2[1][0]

  binding = infos2[2][0]

  author = (soup.find('span', {'class':'author notFaded'})).find('a',{'class':'a-link-normal'}).text

  features = [title, author, n_pages, publishing, language, binding, isbn]

  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)

In [0]:
frames = []

for url in amazon_urls:
  frame = amazon_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [18]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,1808: Como uma rainha,Laurentino Gomes,384,Globo Livros,(Português),Capa Comum,978-8525057518
1,Box - As Melhores Aventuras Do Sítio Do Picapa...,Monteiro Lobato,640,Nova Fronteira,(Português),Capa dura,978-8520943519
2,Cotoco. O Diário De Um Garoto De 13 Anos,John Van de Ruit,390,Intrínseca,(Português),Capa Comum,978-8598078854
3,"Doctor Who: 12 Doutores, 12 histórias",Alex Scarrow,480,Fantástica Rocco,(Português),Capa Comum,978-8568263044
4,Freud (1900) A interpretação dos sonhos: Obras...,Sigmund Freud,736,Companhia das Letras,(Português),Capa dura,978-8535932218
5,Good Omens: Belas Maldições,Neil Gaiman,364,Bertrand Brasil,(Português),Capa Comum,978-8528624021
6,Os Mitos de Cthulhu - Volume Único (Exclusivo ...,Esteban Maroto,92,Pipoca e Nanquim (7 de junho de 2019),(Português),Capa dura,978-8593695261
7,Redes de Computadores e a Internet: Uma Aborda...,Jim Kurose,656,Pearson Universidades,(Português),Capa Comum,978-8581436777
8,Sobotta - Atlas de Anatomia Humana - 3 Volumes,Friedrich Paulsen,1168,Guanabara Koogan,(Português),Capa dura,978-8527732376


In [19]:
analises(amazon_urls, dfs)

Recall: 1.0 
Precision: 1.0 
F_Measure = 1.0 
C = 9 
N = 9 
E = 9


## 2. SARAIVA

In [0]:
def saraiva_single(site):
  soup = bs(open(site),"html.parser")

  infos = soup.find('table')
  infos = [(re.sub('\n|<.*?>|\\n', '///', str(item))) for item in infos]
  infos = [item.split('///') for item in infos]
  infos = [elem for item in infos for elem in item if elem !=  '']

  isbn = [infos[i+1] for i in range(len(infos)) if infos[i] == 'ISBN' or infos[i] == 'ISBN 13' or infos[i] == 'ISBN-13']

  language = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Idioma' or infos[i] == 'Idioma ']

  n_pages = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Páginas' or infos[i] == 'Número de páginas' or infos[i] == 'Número de Páginas']

  binding = [infos[i+1] for info in infos for i in range(len(infos)) if infos[i] == 'Acabamento' or infos[i] == 'Encadernação' or infos[i] == 'Tipo de produto ' or infos[i] == 'Formato']

  author = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Autor']

  publishing = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Editora']

  title = soup.find('div', {'class':'product-review'}).h1.text
    
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)

In [0]:
frames = []

for url in saraiva_urls:
  frame = saraiva_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [22]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Alice No País Das Maravilhas (Classic Edition),"Carroll,Lewis",208,,Português,Capa dura,9788594541758
1,Steve Jobs - A Biografia,"Isaacson,Walter, Isaacson,Walter",624,,Português,Brochura,9788535919714
2,Vade Mecum Compacto - Brochura - 21ª Ed. 2019,Editora Saraiva,2127,,Português,Brochura,9788553603244
3,It - A Coisa,"King,Stephen",1104,,Português,Brochura,9788560280940
4,Como Fazer Amigos e Influenciar Pessoas,,264,,Português,Brochura,9788504018028
5,O Guia do Mochileiro Das Galáxias - Série o Mo...,"Adams,Douglas",208,,Português,Brochura,9788599296578
6,"O Livrinho do Bebê - Animais, Palavras, Contrá...","Books,Yoyo",128,,Português,Brochura,9789463041706
7,Star Wars - O Arquivo Rebelde,"Wallace,Daniel",176,,Português,Brochura,9788528622973


In [23]:
analises(saraiva_urls, dfs)

Recall: 0.0 
Precision: 0.0 
F_Measure = 0 
C = 0 
N = 8 
E = 8


##3. LIVRARIA DA TRAVESSA

In [0]:
def travessa_single(site):
  soup = bs(open(url), 'html.parser')
  
  infos = soup.find('div', {'id':'divDados'}).contents
  infos = [(re.sub('\n|<.*?>|:', '', str(info))) for info in infos]
  infos = [info for info in infos if info != '']
  infos
  
  title = [infos[i+1] for i in range(len(infos)) if infos[i] == 'título']

  author = [re.sub('autor |tradutor','  ',(infos[i])) for i in range(len(infos)) if re.match('autor|tradutor',infos[i])]

  n_pages = [infos[i+1] for i in range(len(infos)) if infos[i] == 'páginas']

  binding = [infos[i+1] for i in range(len(infos)) if infos[i] == 'encadernação']

  language = [infos[i+1] for i in range(len(infos)) if infos[i] == 'idioma']

  isbn = [infos[i+1] for i in range(len(infos)) if infos[i] == 'isbn']

  publishing = soup.find('span', {'id': 'lblNomProdutor'}).text.split(':')[1]
    
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)

In [0]:
frames = []

for url in travessa_urls:
  frame = travessa_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [26]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,A COR DA MAGIA,Terry Pratchett,232,Conrad,Português,Brochura,9788587193391
1,CYNDI MINHA HISTORIA,Cyndi Lauper | Jancee Dunn Aline Naomi Sas...,352,Belas-Letras,Português,Brochura,9788581744988
2,D. PEDRO II A HISTORIA NAO CONTADA - O ULTIMO ...,Paulo Rezzutti,576,Leya,Português,Brochura,9788577346776
3,JAPONES PARA LEIGOS,Eriko Sato Samantha Batista,416,Alta Books,Português,Brochura,9788550802336
4,LONELY PLANET ARGENTINA (INCLUI URUGUAI),,640,Globo Livros,Português,Brochura,9788525068125
5,MADELINE FINN E OS CÃES DO ABRIGO,Lisa Papp Lenice Bueno,32,Salamandra,Português,Brochura,9788516117979
6,O PODER DA AUTORRESPONSABILIDADE A FERRAMENTA ...,Paulo Vieira,160,Gente,Português,Brochura,9788545202219
7,CRONICAS PARA LER EM QUALQUER LUGAR,Gregorio Duvivier | Maria Ribeiro | Xico Sa,112,Todavia,Português,Brochura,9786580309375
8,SO PARA UM ALIMENTAÇAO SAUDAVEL PARA QUEM MORA...,Rita Lobo,168,Senac SP,Português,Capa dura,9788539629510
9,VIDAS SECAS - 140ªED.(2019),Graciliano Ramos,176,Record,Português,Brochura,9788501114785


In [27]:
analises(travessa_urls, dfs)

Recall: 0.9 
Precision: 0.9 
F_Measure = 0.9 
C = 9 
N = 10 
E = 10


##4. TRAÇA LIVRARIA

In [0]:
def tracal_single(site):
  soup = bs(open(site),"html.parser")

  infos = soup.find_all('ul')
  infos = [info.text.split('\n') for info in infos]
  infos = [info for info in infos]
  infos = [info[i].split(':') for info in infos for i in range(len(info))]

  n_pages = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Páginas' or info[i] == 'Número de páginas']

  binding = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Encadernação' or info[i] == 'Tipo de produto ' or info[i] == 'Formato']

  isbn = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'ISBN' or info[i] == 'ISBN 13' or info[i] == 'ISBN-13']

  infos2 = soup.find('div', {'class':'box-principal-dados'})
  infos2 = [info for info in infos2]
  infos2 = [re.sub('\n|<.*?>','', str(info)).split(":") for info in infos2]
  infos2 = [info[i].strip() for info in infos2 for i in range(len(info)) if info[i] != '']

  language = soup.find('div', {'style':'font-size:18px;line-height:20px;'}).text
  language = re.sub('\n|[L][i][v][r][o][ ][e][m] | (.*?)\s', '', language)

  publishing =infos2[7]

  title = soup.find('div', {'class':'box-principal-dados'}).h1.text

  author = soup.find('div', {'class':'box-principal-dados'}).a.text  
    
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)

In [0]:
frames = []

for url in traca_urls:
  frame = tracal_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [30]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,A Casa Dos Budas Ditosos,João Ubaldo Ribeiro,164,Objetiva,Português,Brochura,9788573022391
1,Cavaleiros Do Zodíaco Vol 1,Masami Kurumada,110,Ano,Português,Brochura,
2,Cem Anos De Solidão,Gabriel Garcia Marquez,368,1967,Português,Brochura,
3,Crianças Famosas: Schubert,Ann Rachlin E Susan Hellard,24,Editora,Português,Grampeado,9788574164632
4,Indiozinhos,Mônica Haibara,12,FTD,Português,Grampeado,
5,Livro Da Esperança,Francisco Cândido Xavier - Emmanuel,240,1984,Português,Brochura,
6,Mozart Don Giovanni - A Vida De Mozart,Lorenzo Da Ponte - Henry Stendhal,200,1991,Português,Brochura,
7,O Colapso Do Universo,Isaac Asimov,210,1987,em(Brasil),Capa Dura,
8,Políticas Territoriais Na Amazônia,Neli Aparecida De Mello,412,2006,Português,Brochura,9798574196205
9,Relações Exteriores Do Brasil Contemporâneo,Danielly Silva Ramos Becard,224,Vozes,Português,Brochura,9788532638861


In [31]:
analises(traca_urls, dfs)

Recall: 0.4 
Precision: 0.4 
F_Measure = 0.4 
C = 4 
N = 10 
E = 10


## 5. CIA DOS LIVROS

In [0]:
def cia_dos_livros_single(site):
  soup = bs(open(site), 'html.parser')

  title = soup.find('div', {'class':'detailProduct'}).h1.text

  author = soup.find('dl', {'class':'autor-editora'}).dd.contents[0].strip()

  infos = soup.find('dl', {'id':'bookData'})
  infos_list = [info for info in infos]
  infos_list = [re.sub('\n|<.*?>|:|\t', '', str(info)) for info in infos_list]
  infos_list = [item for item in infos_list if item != '']
  
  language = [infos_list[i+1] for i in range(len(infos_list)) if infos_list[i] == 'Idioma'][0]

  n_pages = [infos_list[i+1] for i in range(len(infos_list)) if infos_list[i] == 'Número de páginas'][0]

  binding = [infos_list[i+1] for i in range(len(infos_list)) if infos_list[i] == 'Encadernação'][0]

  publishing = [infos_list[i+1] for i in range(len(infos_list)) if infos_list[i] == 'Marca'][0]

  isbn = [infos_list[i+1] for i in range(len(infos_list)) if infos_list[i] == 'ISBN' or infos_list[i] == 'ISBN 13' or infos_list[i] == 'ISBN-13' or infos_list[i] == 'ISBN13'][0]

    
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)

In [0]:
frames = []

for url in cia_urls:
  frame = cia_dos_livros_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [34]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Afirmar-se com Nietzsche,Balthasar Thomass,208,Vozes Nobilis,Português,Brochura,9788532660480
1,"Audácia Dessa Mulher, A",Ana Maria Machado,408,Alfaguara,Português,Brochura,9788556520876
2,Avaliação: Mito e Desafio,Jussara Hoffmann,158,MEDIACAO,Português,Brochura,9788577061273
3,Breve História do Feminismo no Contexto Euro-a...,Antje Schrupp,88,EDGARD BLUCHER,Português,Brochura,9788521214038
4,"Corpo Fala, O: A Linguagem Silenciosa da Comun...",Pierre Weil,287,VOZES,Português,Brochura,9788532602084
5,"Cura do Ciúme, A: Aprenda a Confiar, Supere a ...",Robert L. Leahy,192,Artmed,Português,Brochura,9788582715550
6,Dom Casmurro - Coleção Grandes Mestres da Lite...,Machado De Assis,192,LAROUSSE,Português,Brochura,9788581863009
7,Em Busca do Texto Perfeito: Questões Contempor...,Ana Elisa Ribeiro,96,GULLIVER,Português,Brochura,9788565432436
8,Manual do Arquiteto Descalço - Capa Dura,Johan Van Lengen,720,B4 Editores,Português,Capa dura,9788565358668
9,"Feminismo É Feminino?, O",Maíra Marcondes Moreira,204,Annablume,Português,Brochura,9788539109456


In [35]:
analises(cia_urls, dfs)

Recall: 1.0 
Precision: 1.0 
F_Measure = 1.0 
C = 12 
N = 12 
E = 12


##6. EXTRA

In [0]:
def extra_single(site):
  soup = bs(open(site), 'html.parser')
  
  title = soup.find('div', {'class':'produtoNome'}).h1.b.text

  author = soup.find('dl', {'class':'Autor'}).contents
  author = [re.sub('\n|<.*?>', '', str(item)) for item in author]
  author = [item.strip() for item in author if item != '']
  author = [author[i+1] for i in range(len(author)) if author[i] == 'Autor'][0]

  infos = soup.find_all('div', {'class':'wrp'})
  infos = [(re.sub('\n|<.*?>|\\n', '', str(item))).split() for item in infos][1]

  language = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Idioma'][0]

  n_pages = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Páginas'][0]

  publishing = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Editora'][0]

  binding = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Acabamento'][0]

  isbn = [infos[i+1] for i in range(len(infos)) if infos[i] == 'ISBN 13' or infos[i] == 'ISBN' or infos[i] == 'ISBN-13'][0]
          
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)

In [0]:
frames = []

for url in extra_urls:
  frame = extra_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [38]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Aromaterapia e as Emoções: Como Usar Óleos Ess...,Shirley Price,322,Bertrand,Português,Brochura,9788528608519
1,Livro - Once Upon A Time - Irmãos Grimm,Irmãos Grimm,272,Planeta,Português,Brochura,9788542203080
2,Livro - Os Miseráveis - Walcyr Carrasco,Walcyr Carrasco; Victor Hugo,214,Moderna,Português,Brochura,8516079732
3,Livro - Panelinha - Receitas Que Funcionam - R...,Rita Lobo,400,Senac,Português,Brochura,8539602776
4,Livro - A Pequena Filosofia da Mafalda - Vamos...,Joaquim Salvador Lavado (Quino),36,Martins,Português,Brochura,9788580632156
5,Livro - A Seleção: 35 Garotas e uma Coroa - Vo...,Kiera Cass,368,Seguinte,Português,Brochura,9788565765015
6,Livro - Como Eu Era Antes de Você - Jojo Moyes,Jojo Moyes,320,Intrínseca,Português,Brochura,8580573297
7,Livro - Elas Podem... e Devem: o Livro que vai...,Flaviane Brandemberg,114,Ser,Português,Brochura,8563178393
8,Livro - Em Busca de Sentido: Um Psicólogo no C...,Viktor E. Frankl,186,Vozes,Português,Brochura,8532606261
9,"Livro - Eu, Fernando Pessoa: em Quadrinhos - S...",Susana Ventura,72,Peirópolis,Português,Brochura,9788575963050


In [39]:
analises(extra_urls, dfs)

Recall: 1.0 
Precision: 1.0 
F_Measure = 1.0 
C = 10 
N = 10 
E = 10


##7. CULTURA

In [0]:
def cultura_single(site):
  soup = bs(open(site),"html.parser")

  title = soup.find('h1', {'class':'title'}).text

  author = soup.find('ul', {'class':'info'}).a.text

  infos = soup.find('ul', {'id':'product-list-detail'}).find_all('li')
  infos = [(re.sub('\n|<.*?>|\\n|:', '', str(item))).split() for item in infos] 

  isbn = [info[i+1] for info in infos  for i in range(len(info)) if info[i] == 'ISBN'][0]

  language = [info[i+1] for info in infos  for i in range(len(info)) if info[i] == 'Idioma'][0]

  n_pages = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Páginas'][0]

  publishing = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Editora'][0]

  binding = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Encadernação'][1]
    
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)

In [0]:
frames = []

for url in cultura_urls:
  frame = cultura_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [42]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,MAIS ESPERTO QUE O DIABO,"HILL, NAPOLEON",200,CDG,PORTUGUÊS,BROCHURA,8568014003
1,O CAIBALION,TRES INICIADOS,128,PENSAMENTO,PORTUGUÊS,BROCHURA,8531500710
2,O PODER DO HÁBITO,"DUHIGG, CHARLES",408,OBJETIVA,PORTUGUÊS,BROCHURA,8539004119
3,QUEM PENSA ENRIQUECE,"HILL, NAPOLEON",248,FUNDAMENTO,PORTUGUÊS,BROCHURA,8576766302
4,SOCIEDADE DO CANSAÇO,"HAN, BYUNG-CHUL",136,VOZES,PORTUGUÊS,BROCHURA,8532649963
5,GESTALT-TERAPIA - FUNDAMENTOS EPISTEMOLOGICOS,"FRAZAO, LILIAN MEYER",184,SUMMUS,PORTUGUÊS,BROCHURA,8532309089
6,INTRODUÇAO A ALGEBRA LINEAR COM APLICAÇOES,"KOLMAN, BERNARD",684,LTC,PORTUGUÊS,BROCHURA,8521614780
7,O DIÁRIO DE ANNE FRANK,"FRANK, ANNE",224,PRINCIPIS,PORTUGUÊS,BROCHURA,8594318286
8,O MUNDO SEGUNDO FELIPE NETO,"NETO, FELIPE",128,PIXEL,PORTUGUÊS,CAPA,8555462096
9,O PEQUENO PRINCIPE (LIVRO DE BOLSO),"SAINT-EXUPERY, ANTOINE DE",96,CAMINHO,PORTUGUÊS,BROCHURA,8589987337


In [43]:
analises(cultura_urls, dfs)

Recall: 1.0 
Precision: 1.0 
F_Measure = 1.0 
C = 12 
N = 12 
E = 12


## 8. DISAL

In [0]:
def disal_single(site):
  soup = bs(open(site), 'html.parser')

  info_container = soup.find('div', {'class':'book-title'}).contents
  info_container = [re.sub('\n|<.*?>', '', str(info)) for info in info_container]
  info_container = [info.strip() for info in info_container if info != '']

  author = info_container[1]

  title = info_container[0]

  publishing = info_container[2]

  infos = soup.find('table', {'class':'table table-striped'}).find_all('tr')
  infos = [(re.sub('\n|<.*?>|\\n', '', str(item))).split(':') for item in infos]

  isbn = [info[i+1] for info in infos  for i in range(len(info)) if info[i] == 'ISBN' or info[i] == 'ISBN 13'][0]

  language = [info[i+1] for info in infos  for i in range(len(info)) if info[i] == 'Idioma' or info[i] == 'Idioma '][0]

  n_pages = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Páginas' or info[i] == 'Número de páginas'][0]

  binding = [info[i+1] for info in infos for i in range(len(info)) if info[i] == 'Encadernação' or info[i] == 'Tipo de produto '][0]
    
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)


In [0]:
frames = []

for url in disal_urls:
  frame = disal_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [46]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,"Ambiente E Os Processos De Maturacao, O","Winnicott, Dona",268,Artmed,Português,Livro,9788573074567
1,Autorretrato E Outras Crônicas,"Andrade, Drummond De Andrade",256,Record,Português,Livro,9788501112330
2,Aventuras Na Netoland Com Luccas Neto,"Neto, Luccas",64,Pixel Media (nova Fronteira),Português,Livro,9788555461002
3,Exames Laboratoriais E Diagnosticos Em Enferma...,"Fischbach, Frances",732,Guanabara (grupo Gen),Português,Livro,9788527726634
4,Fisiologia Endocrina (lange) - 4º Ed,"Molina, Patricia E.",309,Mcgraw Hill (artmed),Português,Livro,9788580553918
5,Gestao Hospitalar - Para Uma Administracao Eficaz,"Malagon-londono, Gustavo",612,Guanabara (grupo Gen),Português,Livro,9788527733298
6,Gramatica Da Lingua Portuguesa Para Leigos,"Schlee, Magda Bahia",248,Alta Books,Português,Livro,9788550803395
7,Introducao A Analise Matematica,"Avila, Geraldo",260,Edgard Blucher,Português,Livro,9788521201687
8,Michaelis Minidicionario Alemao - Alemao-portu...,Melhoramentos,480,Melhoramentos,Alemão-português / português-alemão,Livro,9788506078563
9,Michaelis Minidicionario Espanhol - Espanhol-p...,Melhoramentos,462,Melhoramentos,Português - Espanhol,Livro,9788506078532


In [47]:
analises(disal_urls, dfs)

Recall: 1.0 
Precision: 1.0 
F_Measure = 1.0 
C = 11 
N = 11 
E = 11


##9. AMERICANAS

In [0]:
def americanas_single(site):
  soup = bs(open(site), 'html.parser')

  infos = soup.find_all('table')
  infos = [info.find_all('td') for info in infos]
  infos = [info.text for info in infos[0]]

  isbn = [infos[i+1] for i in range(len(infos)) if infos[i] == 'ISBN' or infos[i] == 'ISBN 13' or infos[i] == 'ISBN-13']

  language = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Idioma' or infos[i] == 'Idioma ']

  n_pages = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Páginas' or infos[i] == 'Número de páginas']

  binding = [infos[i+1] for info in infos for i in range(len(infos)) if infos[i] == 'Encadernação' or infos[i] == 'Tipo de produto ' or infos[i] == 'Formato']

  title = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Título']

  author = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Autor']

  publishing = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Editora']

    
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)

In [0]:
frames = []

for url in americanas_urls:
  frame = americanas_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [50]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,A cinco passos de você,Rachael Lippincott,288,Globo Alt,Português,Livro brochura,9788525067425
1,A corrente,Adrian McKinty,378,Record,Português,Livro brochura,9788501117656
2,As 13 maldições (Vol. 2),Michelle Harrison,434,Bertrand Brasil,Português,Livro brochura,9788528616668
3,Crônicas Saxônicas: A guerra do lobo (Vol. 11),Bernard Cornwell,378,Record,Português,Livro brochura,9788501117359
4,Eu sou Eric Zimmerman - Volume 2,Megan Maxwell,400,Essência,Português,Livro brochura,9788542217209
5,Herdeiro de Sevenwaters - Vol. 4,Juliet Marillier,496,Butterfly,Português,Livro,9788568674055
6,História da filosofia cristã,"Etienne Gilson, Philotheus Boehner",584,Editora Vozes,Português,Livro brochura,9788532607379
7,Medicina Dos Horrores: A história de Joseph Li...,Lindsey Fizharris,320,Intrínseca,Português,Livro Capa Dura,9788551005224
8,O cadete e o capitão,Luiz Maklouf Carvalho,256,Todavia,Português,Livro brochura,9786580309351
9,Paixão libertadora,Sophie Jackson,352,Editora Arqueiro,Português,Livro brochura,9788580415421


In [51]:
analises(americanas_urls, dfs)

Recall: 1.0 
Precision: 1.0 
F_Measure = 1.0 
C = 10 
N = 10 
E = 10


## 10. SUBMARINO

In [0]:
def submarino_single(site):
  soup = bs(open(site), 'html.parser')

  infos = soup.find_all('table')
  infos = [info.find_all('td') for info in infos]
  infos = [info.text for info in infos[0]]

  isbn = [infos[i+1] for i in range(len(infos)) if infos[i] == 'ISBN' or infos[i] == 'ISBN 13' or infos[i] == 'ISBN-13']

  language = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Idioma' or infos[i] == 'Idioma ']

  n_pages = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Páginas' or infos[i] == 'Número de páginas']

  binding = [infos[i+1] for info in infos for i in range(len(infos)) if infos[i] == 'Encadernação' or infos[i] == 'Tipo de produto ' or infos[i] == 'Formato']

  title = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Título']

  author = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Autor']

  publishing = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Editora']

    
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)

In [0]:
frames = []

for url in submarino_urls:
  frame = submarino_single(url)
  frames.append(frame)

dfs = pd.concat(frames,ignore_index=True)

In [54]:
dfs

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,Luccas Neto em os aventureiros,Luccas Neto,64,Pixel,Português,Livro brochura,9788555461927
1,A dama mais apaixonada,"Julia Quinn, Eloisa James, Connie Brockway",288,Editora Arqueiro,Português,Livro brochura,9788530600211
2,O Labirinto Do Fauno,"Guillermo del Toro, Cornelia Funke",320,Intrínseca,Português,Livro Capa Dura,9788551005194
3,ABSOLUTE SANDMAN VOL. 1,Neil Gaiman,88,Panini Books,Português,Livro Capa Dura,9788573516531
4,,Elizabeth Rudnick,224,,Português,Livro,
5,,"Otomo, Katsuhiro",400,,Português,Livro,
6,,Veronica Roth,448,,Português,Livro,9788579804557
7,,Lauren Blakely,320,,Português,Livro,9788595810846
8,,Bráulio Bessa,160,,Português,Livro,9788543108377
9,Seja Foda! - Black Edition,Caio Carneiro,208,Buzz Editora,Português,Livro Capa Dura,9786580435173


In [55]:
analises(submarino_urls, dfs)

Recall: 0.5 
Precision: 0.5 
F_Measure = 0.5 
C = 5 
N = 10 
E = 10


## One to rule them all

In [0]:
def the_one(site):
  soup = bs(open(site), 'html.parser')

  infos0 = soup.find_all('table')
  infos1 = soup.find_all('ul')
  infos2 = soup.find_all('dl')

  infos = infos0 + infos1 + infos2

  
  infos = [info.find_all('td') for info in infos]
  # infos = [info.find_all('li') for info in infos]
  # infos = [info.find_all('td') for info in infos]
  # infos = [re.sub('\n|<.*?>', '', str(item)) for item in infos]
  infos = [info.text for info in infos[0]]

  isbn = [infos[i+1] for i in range(len(infos)) if infos[i] == 'ISBN' or infos[i] == 'ISBN 13' or infos[i] == 'ISBN-13']

  language = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Idioma' or infos[i] == 'Idioma ']

  n_pages = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Páginas' or infos[i] == 'Número de páginas']

  binding = [infos[i+1] for info in infos for i in range(len(infos)) if infos[i] == 'Encadernação' or infos[i] == 'Tipo de produto ' or infos[i] == 'Formato']

  title = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Título']

  author = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Autor']

  publishing = [infos[i+1] for i in range(len(infos)) if infos[i] == 'Editora']

    
  features = [title, author, n_pages, publishing, language, binding, isbn]
  
  feats = []
  
  for subfeature in features:
    if len(subfeature) == 0:
      feats.append('')
    elif isinstance(subfeature, str):
      feats.append(subfeature)
    elif len(subfeature) == 1:
      feats.append(subfeature[0])
    else:
      feats.append(subfeature[0])

  return create_df(feats)
  # return infos

In [0]:
frames = []

for url in urls:
  try:
    frame = the_one(url)
    frames.append(frame)
  except:
    print("incompatível", url)

df_the_one = pd.concat(frames,ignore_index=True)

In [58]:
df_the_one

,Título,Autor,Nº Páginas,Editora,Idioma,Acabamento,ISBN
0,,,,,,,
1,,,,,,,
2,,,,,,,
3,,,,,,,
4,,,,,,,
5,,,,,,,
6,,,,,,,
7,,,,,,,
8,,,,,,,
9,,,,,,,


In [59]:
analises(urls, df_the_one)

Recall: 0.14705882352941177 
Precision: 0.14705882352941177 
F_Measure = 0.147 
C = 15 
N = 102 
E = 102
